# Informe Recuperos

La finalidad del siguiente informe es dar a conocer ciertas inconsistencias detectadas entre los reportes emitidos por el Sistema Recuperos INVICO. De más está decir que no es posible ni es el objetivo del presente emitir juicio alguno sobre el sistema en sí sino solo de sus reportes siendo estos un output del primero.
Para entender la importancia que reviste el mencionado sistema y sus reportes basta con mencionar que, aproximadamente, el **65% del Activo del INVICO está formado por los saldos de recuperos a cobrar** superando ampliamente a otros ítems como saldos disponibles en banco, obras en ejecución y terminadas entre otros de menor relevancia.
A estos efectos, es necesario contar con información pertinente y confiable elaborada de manera sistémica bajo criterios que se mantenga uniforme a lo largo del tiempo respecto a los **saldos de recuperos a cobrar y las causas de sus variaciones**. 

## Datos iniciales

El único dato que debería modificarse es el relativo al ejercicio sobre el cual se pretende trabajar.

In [ ]:
from invicoctrlpy.recursos.recuperos.recuperos import Recuperos

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('mode.chained_assignment',None)

import itables
itables.options.lengthMenu = [10, 50, 100, -1]
itables.options.maxBytes = 0

import plotly.io as pio
pio.renderers.default = "notebook"

EJERCICIO = '2023'
UPDATE_DB = False
recuperos = Recuperos(ejercicio=EJERCICIO, update_db=UPDATE_DB)

## Recuperos a cobrar

### Evolución del saldo

Una de las primeras cuestiones a analizar es la evolución, en el tiempo, del saldo final de Recuperos a Cobrar. El siguiente gráfico permite reflejar esta cuestión y surge del [Informe de Saldos por Barrio](https://gv.invico.gov.ar/App/Recupero/Informes/InformeSaldosPorBarrio.aspx) Probablemente se trate de uno de uno de los reportes más importantes del Sistema Recuperos. El saldo del mismo y su evolución es reflejado fielmente en los Estados Contables de la institución y el Dpto. Recuperos lo respeta, en igual medida, en su informe final de ejercicio. En cuanto a la razonabilidad del saldo final expuesto, no hay forma de emitir juicio sobre el mismo y no queda otra que tomarlo como válido. 


In [ ]:
recuperos.graficarSaldosBarrio()

Del análisis del gráfico expuesto no quedan dudas que en los ejercicios 2022/2023 se presenta un comportamiento anormal en lo que evolución del saldo se refiere. Un análisis más minucioso de sus causas permitirá dilucidar la cuestión.

### Motivos de variación del saldo

#### Análisis general

Una primera aproximación a esta cuestión la podemos observar en el [Informe Evolucion De Saldos Por Barrio](https://gv.invico.gov.ar/App/Recupero/Informes/InformeEvolucionDeSaldosPorBarrio.aspx). El mismo es similar al [Informe de Saldos por Barrio](https://gv.invico.gov.ar/App/Recupero/Informes/InformeSaldosPorBarrio.aspx) más la adición de dos columnas (amortiza y cambios) que intentan explicar las causas que motivaron la variación en saldos indexando todo por código de barrio. Es aquí donde el **Sistema Recuperos presenta su primera falla**. Si uno procede a realizar la **suma algebraica** de la columnas del informe, **no es posible llegar al saldo final del mismo**. Es más, **en 2023, el sistema solo puede explicar el 6,5% de la diferencia entre el saldo final y el saldo inicial de Recuperos a Cobrar**. La siguiente tabla permite observar la evolución de la difenrencia entre el saldo final expuesto y el calculado por suma algebraica.

In [ ]:
control_suma_saldo_barrio_variacion = recuperos.controlSumaSaldoBarrioVariacion()
itables.show(control_suma_saldo_barrio_variacion, column_filters="footer", dom="lrtip")

#### Análisis particular

A efectos de profundizar en el análisis de causas el reporte ideal es el [Informe de Evolución Saldos Por Motivos](https://gv.invico.gov.ar/App/Recupero/Informes/InformeEvolucionDeSaldosPorMotivos.aspx). Este reporte no es otra cosa que la descomposición de la columna cambios de lo visto en el cuadro anterior. Otro reporte similar es el [Informe Variación Saldos Recuperos a Cobrar](https://gv.invico.gov.ar/App/Recupero/Informes/InformeVariacionSaldosRecuperosACobrar.aspx), cuya única diferencia con el de [Informe de Evolución Saldos Por Motivos](https://gv.invico.gov.ar/App/Recupero/Informes/InformeEvolucionDeSaldosPorMotivos.aspx) es que aplica el criterio de Barrios Incorporados en vez del motivo Entrega de Viviendas. En un próximo apartado se profundizará en esta cuestión.

Volviendo al [Informe de Evolución Saldos Por Motivos](https://gv.invico.gov.ar/App/Recupero/Informes/InformeEvolucionDeSaldosPorMotivos.aspx), ya habíamos adelantado que, para el ejercicio 2023, el mismo solo permite explicar un 6,5% de la variación de saldo de Recuperos a Cobrar. Habiendo hecho esta salvedad, podemos preguntarnos: **¿Cuáles son los cinco (5) motivos que mayor impacto tienen en la variación del Saldo Recupero a Cobrar en el ejercicio bajo análisis?** La tabla siguiente permite reponder esto.

In [ ]:
rank_saldo_motivo_actual = recuperos.rankingSaldoMotivos().head()
itables.show(rank_saldo_motivo_actual, column_filters="footer", dom="lrtip")

Por último, **¿es posible utilizar estos mismos motivos para intentar explicar las variaciones del resto de los ejercicios?** Como puede apreciarse en el gráfico que sigue a continuación, en la mayoría de la casos **si** puesto que la variable **OTROS MOTIVOS se encuentra por debajo del 5% en la mayoría de los años**. Sin embargo, esto no ocurre en **2022 en donde OTROS MOTIVOS es superior al 80%**. La explicación radica en que **en ese año se incorpora el motivo ACTUALIZACIÓN SEMESTRAL el cual no se encuentra presente en 2023 y sea posiblemente la principal causa de las variaciones observadas en este último período**.

In [ ]:
recuperos.graficarPartMotivosBaseOtrosEjercicio()

### Barrios incorporados VS Entrega de Viviendas

Como puede apreciarse en la siguiente tabla, en todos los años existen discrepancias entre los importes de los Barrios Incorporados y los de Entrega de Viviendas. 

In [ ]:
control_barrios_nuevos_vs_entrega_de_viviendas = recuperos.barriosNuevosVsEntregaDeViviendas()
itables.show(control_barrios_nuevos_vs_entrega_de_viviendas, column_filters="footer", dom="lrtip")

Sin embargo, si en vez de indexar por ejercicio se decide trabajar por código de barrio, puede observarse que **las mayores diferencias absolutas se concentran en 5 / 6 barrios**. Por lo tanto, es evidente que existe una diferencia de criterio temporal en el cálculo de los mencionados ítems. 

Siguiendo con el mismo análisis, llama la atención la **diferencia detectada en el código 1348** puesto el motivo entrega de vivienda esta duplicando el valor del informe de barrios incorporados. Es decir, por alguna razón, **el mismo barrio fue dado de alta en ejercicios consecutivos por el mismo valor duplicándose así el mismo. Esto no tiene ningún tipo de lógica**. 

In [ ]:
control_barrios_nuevos_vs_entrega_de_viviendas_por_barrio = recuperos.barriosNuevosVsEntregaDeViviendasPorBarrio()
itables.show(control_barrios_nuevos_vs_entrega_de_viviendas_por_barrio, column_filters="footer", dom="lrtip")

A efectos de lograr una mayor coherencia entre los distintos informes que emite el Sistema Recuperos, **es necesario definir un criterio uniforme en relación a las altas de nuevos barrios**. La pregunta es, ¿cuál criterio es el correcto? Será **aquel que mejor permita explicar la variación entre los saldos de períodos consecutivos**.

## [Resumen Facturado](https://gv.invico.gov.ar/App/Recupero/Informes/ResumenFacturado.aspx) y [Resumen Recaudado](https://gv.invico.gov.ar/App/Recupero/Informes/ResumenRecaudado.aspx)

In [ ]:
recuperos.graficarFacturadoVsRecaudado()

## Recaudado

Una de las primeras cuestiones a observar, en la tabla a continuación, es el hecho de que persisten las diferencias entre lo informado por el [Resumen Facturado](https://gv.invico.gov.ar/App/Recupero/Informes/ResumenFacturado.aspx) y lo realmente recaudado según extractos bancarios (Sistema Indicadores / Sistema de Seguimiento de Cuentas Corrientes). Sin embargo, dichas diferencias observadas son inferiores al 3% (0.12% para el ejercicio bajo análisis).

In [ ]:
control_sist_recuperos_vs_recaudado_real = recuperos.recaudadoSistRecuperosVsRecaudadoReal()
itables.show(control_sist_recuperos_vs_recaudado_real, column_filters="footer", dom="lrtip")

In [ ]:
recuperos.graficarRecaudadoSistRecuperosVsRecaudadoReal()

Algo interesante es que las mayores diferencias, en términos históricos, se detectan en el meses de enero y diciembre, justo con el cambio de ejercicio.

In [ ]:
control_sist_recuperos_vs_recaudado_real_mes = recuperos.recaudadoSistRecuperosVsRecaudadoReal(group_by_month=True)
itables.show(control_sist_recuperos_vs_recaudado_real_mes, column_filters="footer", dom="lrtip")

In [ ]:
recuperos.graficarComposicionRecaudadoActual(6)

In [ ]:
recuperos.graficarAmortizacionRecaudado()

In [ ]:
recuperos.graficarPendAcreditacionRecaudado()

En la siguiente tabla se intenta filtrar aquellos códigos de barrios dado de alta en ejercicios anteriores que tienen una amortización acumulada igual a 0. Estos corresponden a obras que probablemente nunca se cobren y cuya actualización no tiene mucho sentido. 

In [ ]:
alta_barrios_sin_amort = recuperos.altaBarriosSinAmort()
itables.show(alta_barrios_sin_amort, column_filters="footer", dom="lrtip")

Los barrios expuestos en la tabla anterior representan el 12% del total de barrios incorporados, según se muestra a continuación.

In [ ]:
recuperos.graficarAltaBarriosSinAmort()

De todas formas, y como se muestra a continuación, el impacto de las mismas sobre el total de recuperos a cobrar del ejercicio bajo estudio es menor al 2%.

In [ ]:
recuperos.graficarAltaBarriosSinAmort(base_saldo = True)

## EXTRA:

### Excel

In [ ]:
# with pd.ExcelWriter('informe_recuperos.xlsx') as writer:
#     control_actualizacion_semestral.to_excel(writer, sheet_name='control_actualizacion_semestral', index=False)
#     control_mes_gpo_cta_cte.to_excel(writer, sheet_name='control_mes_cta_cte', index=False)
#     siif.to_excel(writer, sheet_name='siif', index=False)
#     sscc.to_excel(writer, sheet_name='sscc', index=False)